In [ ]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
import numqi

tableau = ['#006BA4', '#FF800E', '#ABABAB', '#595959', '#5F9ED1', '#C85200', '#898989', '#A2C8EC', '#FFBC79', '#CFCFCF']

from utils import GeometricCoherenceModel, get_geometric_measure_coherence_sdp
from utils import get_maximally_coherent_state_mixed, get_maximally_coherent_state_mixed_coherence

# Geometric measure of coherence

In [ ]:
dim = 5
p_list = np.linspace(0, 1, 20)
tmp0 = np.stack([get_maximally_coherent_state_mixed(dim, p) for p in p_list])
ret_list_sdp = get_geometric_measure_coherence_sdp(tmp0)
ret_list_analytical = get_maximally_coherent_state_mixed_coherence(dim, p_list)

In [ ]:
fig, ax = plt.subplots()
ax.plot(p_list, ret_list_sdp, '-', label='SDP')
ax.plot(p_list, ret_list_analytical, 'x', label='Analytical')
ax.set_xlabel('p')
ax.set_ylabel('Geometric coherence')
ax.legend()
fig.tight_layout()


In [ ]:
p_list = np.linspace(0, 1, 20)
dim_list = [3, 5, 10, 30]
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
gc_array = np.zeros((len(dim_list), len(p_list)))
gc_analytical_array = np.zeros((len(dim_list), len(p_list)))
for dim in dim_list:
    model = GeometricCoherenceModel(dim, num_term=2*dim, temperature=0.3)
    for i, p in enumerate(tqdm(p_list)):
        model.set_density_matrix(get_maximally_coherent_state_mixed(dim, p))
        theta_opt = numqi.optimize.minimize(model, **kwargs).fun
        with torch.no_grad():
            gc_array[dim_list.index(dim), i] = model(use_temperature=False).item()
    gc_analytical_array[dim_list.index(dim)] = get_maximally_coherent_state_mixed_coherence(dim, p_list)

In [ ]:
fig,ax = plt.subplots()
for i, dim in enumerate(dim_list):
    ax.plot(p_list, gc_analytical_array[i],'o',markerfacecolor='none', color=tableau[i])
    ax.plot(p_list, gc_array[i],'-', color=tableau[i], label=f'$d={dim}$')
ax.legend()
# ax.set_xlabel('p', fontsize=12)
# ax.set_ylabel(r'Geometric measure of coherence',fontsize=12)
# ax.title.set_text('Noisy maximally coherent state')
fig.tight_layout()
#fig.savefig('data/noisy_maximally_coherent_state.pdf')

In [ ]:
p_list = [0.05, 0.1, 0.5, 0.9, 0.95]
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
dim_list = np.arange(2, 51)
err_array = np.zeros((len(dim_list), len(p_list)))
time_array_gd = np.zeros((len(dim_list), len(p_list)))
time_array_sdp = np.zeros((len(dim_list), len(p_list)))
for dim in tqdm(dim_list):
    model = GeometricCoherenceModel(dim, num_term=2*dim, temperature=0.3)
    for i, p in enumerate(p_list):
        model.set_density_matrix(get_maximally_coherent_state_mixed(dim, p))
        t0 = time()
        theta_opt = numqi.optimize.minimize(model, **kwargs).fun
        t1 = time()
        get_geometric_measure_coherence_sdp(get_maximally_coherent_state_mixed(dim, p))
        t2 = time()
        time_array_gd[dim-2, i] = t1-t0
        time_array_sdp[dim-2, i] = t2-t1
        with torch.no_grad():
            err_array[dim-2, i] = np.abs(model(use_temperature=False).item()-get_maximally_coherent_state_mixed_coherence(dim, p))

In [ ]:
# save data in pkl
with open('data/noisy_maximally_coherent_state.pkl', 'wb') as f:
    pickle.dump([err_array, time_array_gd, time_array_sdp], f)

In [ ]:
# load data from pkl
with open('data/noisy_maximally_coherent_state.pkl', 'rb') as f:
    err_array, time_array_gd, time_array_sdp = pickle.load(f)

In [ ]:
fig,ax = plt.subplots()
for i, p in enumerate(p_list):
    ax.plot(dim_list, err_array[:,i],'-', label=f'$p={p}$', color=tableau[i])
ax.legend()
# ax.set_xlabel(r'Dimension $d$', fontsize=12)
# ax.set_ylabel(r'Numerical error',fontsize=12)
# ax.title.set_text('Noisy maximally coherent state')
fig.tight_layout()
#fig.savefig('data/noisy_maximally_coherent_state_error.pdf')

In [ ]:
fig,ax = plt.subplots()
for i, p in enumerate(p_list):
    ax.plot(dim_list, time_array_gd[:,i],'-', label=f'$p={p}$', color=tableau[i])
    ax.plot(dim_list, time_array_sdp[:,i],'--', color=tableau[i])
ax.legend()
# ax.set_xlabel(r'Dimension $d$', fontsize=12)
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
# ax.set_ylabel(r'Computational time',fontsize=12)
# ax.title.set_text('Noisy maximally coherent state')
fig.tight_layout()
# fig.savefig('data/noisy_maximally_coherent_state_time.pdf')

# Entanglement of formation

In [ ]:
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
dA = 2
dB = 3
rank = 6
dim = dA*dB
t_polar = []
t_exp = []
t_euler = []
for _ in range(10):
    rho = numqi.random.rand_density_matrix(dim, k=rank)
    model= numqi.entangle.EntanglementFormationModel(dA, dB, num_term=2*dim, method='euler', euler_with_phase=True)
    model.set_density_matrix(rho)
    t0 = time()
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    t_euler.append(time()-t0)
    model= numqi.entangle.EntanglementFormationModel(dA, dB, num_term=2*dim, method='so-exp', euler_with_phase=False)
    model.set_density_matrix(rho)
    t0 = time()
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    t_exp.append(time()-t0)
    model= numqi.entangle.EntanglementFormationModel(dA, dB, num_term=2*dim, method='polar', euler_with_phase=False)
    model.set_density_matrix(rho)
    t0 = time()
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    t_polar.append(time()-t0)
# print avg time (3 decimals) for each method
print(f'polar: {np.mean(t_polar):.3f}')
print(f'so-exp: {np.mean(t_exp):.3f}')
print(f'euler: {np.mean(t_euler):.3f}')

In [ ]:
alpha_list = np.linspace(0, 1, 50)
dim_list = [3, 4, 5, 6]
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
eof_array = np.zeros((len(dim_list), len(alpha_list)))
for dim in dim_list:
    model= numqi.entangle.EntanglementFormationModel(dim, dim, num_term=2*dim*dim)
    for i, alpha in enumerate(tqdm(alpha_list)):
        model.set_density_matrix(numqi.state.Werner(dim, alpha))
        theta_optim = numqi.optimize.minimize(model, **kwargs)
        eof_array[dim_list.index(dim), i] = theta_optim.fun

In [ ]:
fig, ax = plt.subplots()

for i, dim in enumerate(dim_list):
    alpha_list_filter = alpha_list[alpha_list>1/dim]
    eof_analytical = numqi.state.get_Werner_eof(dim, alpha=alpha_list_filter)
    ax.plot(alpha_list_filter, eof_analytical, 'o', markerfacecolor='none', color=tableau[i])
    ax.plot(alpha_list, eof_array[i],'-', label=f'$d={dim}$', color=tableau[i])
    ax.axvline(1/dim, color=tableau[i], linestyle='--')
ax.legend()
# log scale
ax.set_yscale('log')
# ax.set_xlabel(r'$\alpha$', fontsize=12)
# ax.set_ylabel(r'Entanglement of formation',fontsize=12)
# ax.set_title('Werner state')
fig.tight_layout()
#fig.savefig('data/Werner_state.pdf')

In [ ]:
alpha_list = [0.05, 0.1, 0.5, 0.9, 0.95]
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
dim_list = np.arange(2, 7)
err_array = np.zeros((len(dim_list), len(alpha_list)))
time_array = np.zeros((len(dim_list), len(alpha_list)))

for dim in tqdm(dim_list):
    model= numqi.entangle.EntanglementFormationModel(dim, dim, num_term=2*dim*dim)
    for i, alpha in enumerate(alpha_list):
        model.set_density_matrix(numqi.state.Werner(dim, alpha))
        t0 = time()
        theta_opt = numqi.optimize.minimize(model, **kwargs).fun
        t1 = time()
        time_array[dim-2, i] = t1-t0
        err_array[dim-2, i] = np.abs(numqi.optimize.minimize(model, **kwargs).fun-numqi.state.get_Werner_eof(dim, alpha))

In [ ]:
#save data in pkl
with open('data/werner.pkl', 'wb') as f:
    pickle.dump([err_array, time_array], f)

In [ ]:
# load data from pkl
with open('data/werner.pkl', 'rb') as f:
    err_array, time_array = pickle.load(f)
dim_list = np.arange(2, 7)

fig,ax = plt.subplots()
for i, alpha in enumerate(alpha_list):
    ax.plot(dim_list, err_array[:,i],'-', label=f'$\\alpha={alpha}$', color=tableau[i])
ax.legend()
# ax.set_xlabel(r'Dimension $d$', fontsize=12)
# ax.set_ylabel(r'Numerical error',fontsize=12)
# ax.title.set_text('Werner state')
# set x-axis to integer
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
fig.tight_layout()
#fig.savefig('data/Werner_state_error.pdf')

In [ ]:
fig,ax = plt.subplots()
for i, alpha in enumerate(alpha_list):
    ax.plot(dim_list, time_array[:,i],'-', label=f'$\\alpha={alpha}$', color=tableau[i])
ax.legend()
# ax.set_xlabel(r'Dimension $d$', fontsize=12)
# ax.set_ylabel(r'Computational time',fontsize=12)
# ax.title.set_text('Werner state')
# set x-axis to integer
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
fig.tight_layout()
# fig.savefig('data/Werner_state_time.pdf')

# Linear entropy of entanglement

In [ ]:
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
dA = 3
dB = 3
rank = 9
dim = dA*dB
t_polar = []
t_ppt = []
for _ in range(10):
    rho = numqi.random.rand_density_matrix(dim, k=rank)
    model = numqi.entangle.DensityMatrixLinearEntropyModel([dA,dB], num_ensemble=2*dim, kind='convex')
    model.set_density_matrix(rho)
    t0 = time()
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    t_polar.append(time()-t0)
    t0 = time()
    numqi.entangle.get_linear_entropy_entanglement_ppt(rho, (dA,dB))
    t_ppt.append(time()-t0)
# print avg time (3 decimals) for each method
print(f'polar: {np.mean(t_polar):.3f}')
print(f'ppt: {np.mean(t_ppt):.3f}')


In [ ]:
alpha_list = np.linspace(0, 1, 50)
rho_list = np.stack([numqi.state.get_bes3x3_Horodecki1997(alpha) for alpha in alpha_list])
ret_ppt = numqi.entangle.get_linear_entropy_entanglement_ppt(rho_list, (3,3), use_tqdm=True)

ret_polar = []
model = numqi.entangle.DensityMatrixLinearEntropyModel([3,3], num_ensemble=18, kind='convex')
for rho in tqdm(rho_list):
    model.set_density_matrix(rho)
    ret_polar.append(numqi.optimize.minimize(model, num_repeat=3, tol=1e-14, print_every_round=0).fun)
ret_polar = np.array(ret_polar)

In [ ]:
# save date in pkl
with open('data/linear_entropy_entanglement_ppt.pkl', 'wb') as f:
    pickle.dump(ret_ppt, f)
with open('data/linear_entropy_entanglement_polar.pkl', 'wb') as f:
    pickle.dump(ret_polar, f)

In [ ]:
alpha_list = np.linspace(0, 1, 50)
# load data from pkl
with open('data/linear_entropy_entanglement_ppt.pkl', 'rb') as f:
    ret_ppt = pickle.load(f)
with open('data/linear_entropy_entanglement_polar.pkl', 'rb') as f:
    ret_polar = pickle.load(f)

fig,ax = plt.subplots()
ax.plot(alpha_list, ret_polar, label='GD', linestyle= '-', color=tableau[0])
ax.plot(alpha_list, ret_ppt, label='SDP', linestyle= '--', color=tableau[1])
ax.legend()
# ax.set_xlabel(r'$\alpha$', fontsize=12)
# ax.set_ylabel(r'Linear entropy of entanglement', fontsize=12)
# ax.set_yscale('log')
# set yscale to sceintific notation
ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.set_title('Horodecki state')
fig.tight_layout()
#fig.savefig('data/horodecki.pdf')